In [1]:
import pandas as pd
from cleantext import clean

from nltk.corpus import stopwords
from nltk.probability import FreqDist
import numpy as np

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
def limpar_texto(text):
    
    return clean.clean(text,
                    fix_unicode=True,              
                    to_ascii=True,                 
                    lower=True,                    
                    no_line_breaks=True,
                    no_urls=True,
                    no_emails=True,     
                    no_phone_numbers=True,
                    no_numbers=True,      
                    no_digits=True,       
                    no_currency_symbols=True, 
                    no_punct=True,          
                    replace_with_punct="",
                    replace_with_url="",
                    replace_with_email="",
                    replace_with_phone_number="",
                    replace_with_number="",
                    replace_with_digit="",
                    replace_with_currency_symbol="",
                    lang="pt-br"
                )
    
def plot_frequencia_tokens(tokens):
    fd_words = FreqDist(tokens)
    fd_words.plot(20)
    
def remover_stopwords(text, personal_stop=[]):
    
    portugues_stops = stopwords.words('portuguese')

    text = text.split(' ')
    text = [x for x in text if x not in portugues_stops]
    text = [x for x in text if x not in personal_stop]
    return ' '.join(text)

def get_most_frequent_words(words, total=100):
    fd_words = FreqDist(words)

    words_to_remove = []
    counter = 0
    for word, i in fd_words.most_common():
        words_to_remove.append(word)
        counter += 1
        if counter > total:
            break
    
    return words_to_remove
    
def contador_palavras(texts):
    total = 0
    count_texts = 0
    max = 0
    min = 0
    for text in texts:
        count_texts += 1
        total_palavras = len(text.split(' '))
        total += total_palavras
        if total_palavras > max:
            max = total_palavras
        
        if min > total_palavras or min == 0:
            min = total_palavras

    return total, total/count_texts, max, min

def listar_palavras_do_Series(series):
    words_to_remove = []
    for text in series:
        for word in text.split(' '):
            words_to_remove.append(word)

    return words_to_remove



In [3]:
df_artigos = pd.read_csv('articles.csv')
df_novos_artigos = pd.read_excel("new_scrapped_articles/fixed_articles.xlsx")


In [4]:
df_artigos.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [ ]:
df_artigos.date = df_artigos.date.astype(str)

In [5]:
df_novos_artigos[df_novos_artigos.date == 'None'].shape, df_novos_artigos.shape

((64, 5), (3472, 5))

In [6]:
df_novos_artigos.drop(inplace=True, index=df_novos_artigos[df_novos_artigos.date == 'None'].index)

In [7]:
df_novos_artigos.date = pd.to_datetime(df_novos_artigos.date)

In [8]:
df_novos_artigos.date[1], df_artigos.date[1]

(Timestamp('2016-12-05 00:00:00'), '2017-09-10')

In [9]:
df_artigos.shape

(167053, 6)

In [24]:
new_data = []

for i, item in df_novos_artigos.iterrows():

    new_data.append([
            item.title, 
            item.text,
            item.date.strftime('%Y-%m-%d'),
            str(item.section).lower(),
            np.nan,
            item.url,
        ])



In [25]:
df_novos_artigos_2 = pd.DataFrame(columns=[
            'title', 
            'text',
            'date',
            'category',
            'subcategory',
            'link']
            , data=new_data
            )

In [26]:
df_novos_artigos_2.head()

,title,text,date,category,subcategory,link
0,"Após dois cortes, Petrobras eleva preços da ga...",Mal o consumidor sentiu no bolso o impacto de ...,2016-12-05,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/12/1...
1,Liminar da Justiça de Sergipe suspende venda d...,A 3ª Vara da Justiça Federal de Sergipe determ...,2016-12-05,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/12/1...
2,Dólar recua mais de 1% após protestos pouparem...,O real teve uma das maiores valorizações entre...,2016-12-05,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/12/1...
3,Metade das empresas não tem caixa para pagar d...,A prolongada crise econômica que o país atrave...,2016-12-04,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/12/1...
4,Governo busca executivo experiente para o coma...,Sob pressão do PMDB da Câmara dos Deputados pa...,2016-11-05,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/11/1...


In [27]:
df_new = pd.concat([df_artigos, df_novos_artigos_2], axis=0)
df_new

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...
...,...,...,...,...,...,...
3403,Bilheterias do Pacaembu estarão fechadas no do...,Não haverá venda de ingressos no Pacaembu na m...,2016-03-12,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/03/1...
3404,"Contra Palmeiras, São Paulo quer comprovar mud...","Neste domingo (13), os jogadores do São Paulo ...",2016-03-12,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/03/1...
3405,"Na volta ao Allianz Parque, Palmeiras leva vir...",No primeiro jogo no Allianz Parque após o títu...,2016-02-13,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/02/1...
3406,"Quem não acredita que pule fora, diz Fernando ...",A crise no Palmeiras aumentou ainda mais neste...,2016-03-27,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/03/1...


In [28]:
df_new.date.max(), df_new.date.min()

('2017-10-01', '2015-01-01')

In [29]:
df_new.shape

(170461, 6)

In [30]:
df_new.head()
df_new.category.unique()

array(['poder', 'ilustrada', 'mercado', 'mundo', 'esporte', 'tec',
       'cotidiano', 'ambiente', 'equilibrioesaude', 'sobretudo',
       'colunas', 'educacao', 'tv', 'banco-de-dados', 'opiniao',
       'ciencia', 'paineldoleitor', 'saopaulo', 'ilustrissima',
       'seminariosfolha', 'turismo', 'empreendedorsocial', 'serafina',
       'asmais', 'o-melhor-de-sao-paulo', 'bbc', 'comida', 'musica',
       'folhinha', 'especial', 'treinamento', 'multimidia',
       'cenarios-2017', 'topofmind', 'dw', 'ombudsman', 'contas-de-casa',
       'mulher', '2016', 'guia-de-livros-discos-filmes',
       'treinamentocienciaesaude', 'rfi', 'vice', 'bichos', 'euronews',
       'guia-de-livros-filmes-discos', 'infograficos', '2015',
       'colunistas', 'opinião', 'as mais', 'educação', 'tvfolha ', 'nan',
       'ilustríssima'], dtype=object)

In [32]:
df_artigos.shape, df_new.shape

((167053, 6), (170461, 6))

In [47]:
df_new.drop(index=df_new[df_new.text.isna()].index, inplace=True)
df_artigos.drop(index=df_artigos[df_artigos.text.isna()].index, inplace=True)

In [48]:
df_artigos.shape, df_new.shape

((166288, 6), (169671, 6))

In [49]:
filtro = (df_artigos.text.str.lower().str.contains("etrobr"))
df_artigos[filtro].shape

(10604, 6)

In [50]:
filtro = (df_new.text.str.lower().str.contains("etrobr"))
df_new[filtro].shape

(11353, 6)

In [51]:
df_new = df_new[filtro]

In [52]:
df_new = df_new.drop(df_new[df_new.date.isnull()].index)


In [53]:
df_new['date_str'] = df_new.date.convert_dtypes(convert_string=True)

In [54]:
df_new.shape

(11353, 7)

In [55]:
df_new.head()

,title,text,date,category,subcategory,link,date_str
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...,2017-09-10
18,Bolsa recua pelo 2º dia com feriado nos EUA; d...,A Bolsa brasileira fechou em baixa nesta segun...,2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...,2017-09-10
21,Moro pede que Lula esclareça se tem os recibos...,O juiz Sergio Moro quer que o ex-presidente Lu...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...,2017-09-10
76,Lula na memória e no coração,"A um ano das eleições de 2018, a candidatura d...",2017-09-10,opiniao,NaN,http://www1.folha.uol.com.br/opiniao/2017/10/1...,2017-09-10
152,Destruindo a ciência brasileira,"Neste domingo (8), ia escrever sobre o prêmio ...",2017-08-10,colunas,marcelogleiser,http://www1.folha.uol.com.br/colunas/marcelogl...,2017-08-10


In [56]:
df_new.text = [limpar_texto(txt) for txt in df_new.text.replace('\n', ' ').replace('\t', ' ')]

In [57]:
contador_palavras(df_new.text)

(6050786, 532.9680260724037, 9666, 11)

In [58]:
df_new.text = [remover_stopwords(txt) for txt in df_new.text]

In [59]:
df_new['date_str'] = pd.to_datetime(df_new.date, errors='coerce')

In [60]:
df_new = df_new.drop(df_new[df_new.date_str.isnull()].index)
df_new.shape

(11353, 7)

In [61]:
df_new = df_new.drop(df_new[df_new.text.isnull()].index)
df_new.shape

(11353, 7)

In [62]:
df_new.to_csv('articles_por_data_sem_merge.csv', sep='\t')

In [63]:
select_header = ['date', 'title', 'text', 'link']

In [64]:
df_dados_etiquetados = pd.read_excel('dados_etiquetados.xlsx', sheet_name='dados', index_col=False)
df_articles_por_data_etiquetados = pd.merge(df_new[select_header], df_dados_etiquetados[['title', 'sobre_petrobras_diretamente']], on=['title'], how='left')

In [ ]:
df_articles_por_data_etiquetados.dropna().to_csv('articles_por_data_etiquetados.csv', sep='\t', encoding='utf-8')

In [ ]:
df_articles_petro = df_new[df_new.text.str.contains('Petrobras')]
df_articles_petro_grouped = df_new.groupby('date')

df_articles_por_data = pd.concat([df_articles_petro_grouped.title.apply(lambda x: ' '.join(x)), df_articles_petro_grouped.text.apply(lambda x: ' '.join(x)), df_articles_petro_grouped.link.apply(lambda x: ' - '.join(x))], axis=1)


In [ ]:
df_articles_por_data_etiquetados.to_csv('articles_per_date.csv', sep='\t', encoding='utf-8')